### Decorator Application: Single Dispatch Generic Functions (Part 2)

In [2]:
from decimal import Decimal
from html import escape

In [3]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{round(a, 2):.2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>'
                for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{html_escape(k)}={htmlize(v)}</li>'
                for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)

In [ ]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_int,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict,
    }
    fn = registry.get(type(arg), registry[object])
    return fn(arg)

In [5]:
htmlize(100)

'100(<i>0x64</i>)'

In [6]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>
